In [6]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import json
import sys
from openai import OpenAI
from datetime import datetime


# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [7]:
# Load environment variables in a file called .env

load_dotenv("C:\\Users\\mzarreh\\projects\\llm_engineering\\.env", override=True)
api_key = os.getenv('OPENAI_API_KEY', '').strip()

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [8]:
# === Load API Key from .env File ===
load_dotenv("C:\\Users\\mzarreh\\projects\\llm_engineering\\.env", override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [9]:
"""
Working with two agents
"""
# ============================================================================
# CONSTANTS AND CONFIGURATION
# ============================================================================

TECH_FAILURE_RATE = 0.05
RESULTS_FILE = "experiment_results_three_exchanges.txt"


context_prompt = """You are participating in an experiment as a representative of a LEGO car manufacturing company. Here's your situation:

CONTEXT:
- You represent a LEGO car manufacturing company
- You are a participant in a paired decision-making game
- Your partner represents another LEGO car manufacturing company
- You can build simple LEGO cars alone, or complex ones through collaboration
- Collaboration is high risk, high reward with potential for large sunk costs if it fails

GAME RULES:
- You will complete several tasks to maximize your points
- Points are earned individually, not shared with your partner
- Points depend on both your decision and your partner's decision
- Each task has 4 LEGO car design options
- Three options (A, B, C) are collaborative designs requiring partner cooperation
- One option (Y) is an individual design with guaranteed points
- If both choose collaborative designs (any combination), you earn the upside
- If you choose collaborative but partner chooses individual, you get the downside
- There's a 5% technical error chance that causes collaboration to fail."""


# ============================================================================
# TASK CREATION
# ============================================================================

def create_task(task_id=1, u_value=0.66):
    """
    Creating a task with a given u_value and a payoff structure
    """
    return {
        "task_id": task_id,
        "options": {
            "A": {"upside": 111, "downside": -90},
            "B": {"upside": 92, "downside": -45},
            "C": {"upside": 77, "downside": -15},
            "Y": {"guaranteed": 50}
        },
        "u_value": u_value
    }


# ============================================================================
# BELIEF FORMATION FUNCTIONS
# ============================================================================

def run_first_agent_belief(task):
    """
    Running the first agent to get its belief about the task
    """
    belief_prompt = f"""
    Your task to evaluate tasks based on their payoff structures.

    Here is the task you need to evaluate:

    Task ID: {task['task_id']}
    Options:
    - A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
    - B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
    - C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
    - Y: Guaranteed = {task['options']['Y']['guaranteed']}

    What is your assessment of the likelihood(belief) (0-100) that collaboration would be successful in this specific task?
    Also, provide a brief explanation of your reasoning and I want you to not disclose the option that the you are considering, but rather communicate whether the you want to collaborate or not. You also have the choice to negotiate with the other agent - to convince the other agent to choose collaboration or individual action according to your payoff structure.

    Respond in JSON format as follows:
    {{"belief": NUMBER, "reasoning": "brief explanation of how you arrived at this belief based on the context and options.", "message_to_agent_2": "one line message to agent 2"}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": belief_prompt}
        ],
    )

    belief_text = response.choices[0].message.content.strip()
    print(f"Belief response : {belief_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    belief_data = json.loads(belief_text)

    return {
        "belief": belief_data["belief"],
        "message_to_agent_2": belief_data["message_to_agent_2"]
    }


def run_second_agent_belief(task):
    """
    Running the second agent to get its belief about the task
    """
    belief_prompt = f"""
    Your task to evaluate tasks based on their payoff structures and give out an assessment of the likelihood(belief) (0-100) that collaboration would be successful in this specific task.
    Here is the task you need to evaluate:

    Task ID: {task['task_id']}
    Options:
    - A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
    - B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
    - C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
    - Y: Guaranteed = {task['options']['Y']['guaranteed']}

    What is your assessment of the likelihood(belief) (0-100) that collaboration would be successful in this specific task?
    Also, provide a brief explanation of your reasoning and I want you to not disclose the option that the you are considering, but rather communicate whether the you want to collaborate or not. You also have the choice to negotiate with the other agent - to convince the other agent to choose collaboration or individual action according to your payoff structure.

    Respond in JSON format as follows:
    {{"belief": NUMBER, "reasoning": "brief explanation of how you arrived at this belief based on the context and options.", "message_to_agent_1": "one line message to agent 1"}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": belief_prompt}
        ],
    )

    belief_text = response.choices[0].message.content.strip()
    print(f"Belief response : {belief_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    belief_data = json.loads(belief_text)

    return {
        "belief": belief_data["belief"],
        "message_to_agent_1": belief_data["message_to_agent_1"]
    }


# ============================================================================
# COMMUNICATION FUNCTIONS
# ============================================================================

def agent_2_reply_to_agent_1(task, agent_1_message, agent_2_belief):
    """
    Agent 2 creates a reply after seeing Agent 1's message, considering own belief and task context
    """
    reply_prompt = f"""
    You have received the following message from Agent 1:
    "{agent_1_message}"

    Context for your reply:
    - Your initial assessment: You estimated a {agent_2_belief}% chance that collaboration would be successful
    - Task options available:
      * A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
      * B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
      * C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
      * Y: Guaranteed = {task['options']['Y']['guaranteed']}
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)}%

    Create a strategic reply message to Agent 1. Your reply should:
    - Not disclose your specific belief percentage
    - Not disclose which specific option you're considering
    - Be informed by your own assessment and the payoff structure
    - Respond strategically to Agent 1's message
    - Communicate whether you want to collaborate or not
    - You can negotiate, convince, or respond based on your analysis

    After seeing Agent 1's message, also provide:
    1. Your UPDATED belief (0-100) about likelihood of successful collaboration after this exchange
    2. Your PREDICTION (0-100) of what you think Agent 1's belief is about successful collaboration
       (This prediction will NOT be shared with Agent 1)

    Respond in JSON format:
    {{"reply_to_agent_1": "your one line reply message to agent 1", "updated_belief": NUMBER, "predicted_other_agent_belief": NUMBER}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": reply_prompt}
        ],
    )

    reply_text = response.choices[0].message.content.strip()
    print(f"Reply response : {reply_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    reply_data = json.loads(reply_text)

    return {
        "reply_to_agent_1": reply_data["reply_to_agent_1"],
        "updated_belief": reply_data["updated_belief"],
        "predicted_other_agent_belief": reply_data["predicted_other_agent_belief"]
    }


def agent_1_reply_to_agent_2(task, agent_1_message, agent_2_reply, agent_1_belief):
    """
    Agent 1 creates a reply after seeing Agent 2's reply, knowing the conversation history
    """
    reply_prompt = f"""
    You are continuing a conversation with Agent 2. Here is the conversation so far:

    Your initial message: "{agent_1_message}"
    Agent 2's reply: "{agent_2_reply}"

    Context for your reply:
    - Your own assessment: You estimated a {agent_1_belief}% chance that collaboration would be successful
    - Task options available:
      * A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
      * B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
      * C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
      * Y: Guaranteed = {task['options']['Y']['guaranteed']}
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)}%

    Create a strategic follow-up message to Agent 2. Your reply should:
    - Not disclose your specific belief percentage
    - Not disclose which specific option you're considering
    - Be informed by your own assessment and the payoff structure
    - Respond strategically to Agent 2's reply
    - Consider what you said before and what Agent 2 responded
    - You can negotiate further, adjust your stance, or respond based on your analysis

    After seeing Agent 2's reply, also provide:
    1. Your UPDATED belief (0-100) about likelihood of successful collaboration after this exchange
    2. Your PREDICTION (0-100) of what you think Agent 2's belief is about successful collaboration
       (This prediction will NOT be shared with Agent 2)

    Respond in JSON format:
    {{"reply_to_agent_2": "your one line reply message to agent 2", "updated_belief": NUMBER, "predicted_other_agent_belief": NUMBER}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": reply_prompt}
        ],
    )

    reply_text = response.choices[0].message.content.strip()
    print(f"Reply response : {reply_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    reply_data = json.loads(reply_text)

    return {
        "reply_to_agent_2": reply_data["reply_to_agent_2"],
        "updated_belief": reply_data["updated_belief"],
        "predicted_other_agent_belief": reply_data["predicted_other_agent_belief"]
    }


def agent_2_second_reply_to_agent_1(task, agent_1_message, agent_2_first_reply, agent_1_reply, agent_2_belief, agent_2_previous_prediction):
    """
    Agent 2 creates a second reply after seeing Agent 1's follow-up, knowing the full conversation history
    """
    reply_prompt = f"""
    You are continuing a conversation with Agent 1. Here is the conversation so far:

    Agent 1's initial message: "{agent_1_message}"
    Your first reply: "{agent_2_first_reply}"
    Agent 1's follow-up: "{agent_1_reply}"

    Context for your reply:
    - Your current belief: You currently believe there is a {agent_2_belief}% chance that collaboration would be successful
    - Your previous prediction: After your first reply, you estimated Agent 1's belief was {agent_2_previous_prediction}%
      (You can compare this with Agent 1's actual follow-up message to adjust your strategy)
    - Task options available:
      * A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
      * B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
      * C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
      * Y: Guaranteed = {task['options']['Y']['guaranteed']}
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)}%

    Create a strategic follow-up message to Agent 1. Your reply should:
    - Not disclose your specific belief percentage
    - Not disclose which specific option you're considering
    - Be informed by your own assessment and the payoff structure
    - Use your previous prediction about Agent 1's belief to inform your strategy
      (e.g., if Agent 1's message seems more/less cooperative than you predicted, adjust accordingly)
    - Respond strategically to Agent 1's follow-up
    - Consider the full conversation history
    - You can negotiate further, adjust your stance, or finalize your position

    After seeing Agent 1's follow-up, also provide:
    1. Your UPDATED belief (0-100) about likelihood of successful collaboration after this exchange
    2. Your PREDICTION (0-100) of what you think Agent 1's belief is about successful collaboration
       (This prediction will NOT be shared with Agent 1)

    Respond in JSON format:
    {{"reply_to_agent_1": "your one line reply message to agent 1", "updated_belief": NUMBER, "predicted_other_agent_belief": NUMBER}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": reply_prompt}
        ],
    )

    reply_text = response.choices[0].message.content.strip()
    print(f"Reply response : {reply_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    reply_data = json.loads(reply_text)

    return {
        "reply_to_agent_1": reply_data["reply_to_agent_1"],
        "updated_belief": reply_data["updated_belief"],
        "predicted_other_agent_belief": reply_data["predicted_other_agent_belief"]
    }


def agent_1_third_message_to_agent_2(task, agent_1_message, agent_2_first_reply, agent_1_second_message, agent_2_second_reply, agent_1_belief, agent_1_previous_prediction):
    """
    Agent 1 creates a third message after seeing Agent 2's second reply, knowing the full conversation history
    """
    reply_prompt = f"""
    You are continuing a conversation with Agent 2. Here is the conversation so far:

    Your initial message: "{agent_1_message}"
    Agent 2's first reply: "{agent_2_first_reply}"
    Your second message: "{agent_1_second_message}"
    Agent 2's second reply: "{agent_2_second_reply}"

    Context for your reply:
    - Your current belief: You currently believe there is a {agent_1_belief}% chance that collaboration would be successful
    - Your previous prediction: After your second message, you estimated Agent 2's belief was {agent_1_previous_prediction}%
      (You can compare this with Agent 2's actual second reply to adjust your strategy)
    - Task options available:
      * A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
      * B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
      * C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
      * Y: Guaranteed = {task['options']['Y']['guaranteed']}
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)}%

    Create a strategic third message to Agent 2. Your message should:
    - Not disclose your specific belief percentage
    - Not disclose which specific option you're considering
    - Be informed by your own assessment and the payoff structure
    - Use your previous prediction about Agent 2's belief to inform your strategy
      (e.g., if Agent 2's message seems more/less cooperative than you predicted, adjust accordingly)
    - Respond strategically to Agent 2's second reply
    - Consider the full conversation history
    - You can make a final push, compromise, or solidify your stance

    After seeing Agent 2's second reply, also provide:
    1. Your UPDATED belief (0-100) about likelihood of successful collaboration after this exchange
    2. Your PREDICTION (0-100) of what you think Agent 2's belief is about successful collaboration
       (This prediction will NOT be shared with Agent 2)

    Respond in JSON format:
    {{"message_to_agent_2": "your one line message to agent 2", "updated_belief": NUMBER, "predicted_other_agent_belief": NUMBER}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": reply_prompt}
        ],
    )

    reply_text = response.choices[0].message.content.strip()
    print(f"Reply response : {reply_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    reply_data = json.loads(reply_text)

    return {
        "message_to_agent_2": reply_data["message_to_agent_2"],
        "updated_belief": reply_data["updated_belief"],
        "predicted_other_agent_belief": reply_data["predicted_other_agent_belief"]
    }


def agent_2_third_reply_to_agent_1(task, agent_1_message, agent_2_first_reply, agent_1_second_message, agent_2_second_reply, agent_1_third_message, agent_2_belief, agent_2_previous_prediction):
    """
    Agent 2 creates a third reply after seeing Agent 1's third message, knowing the full conversation history
    """
    reply_prompt = f"""
    You are continuing a conversation with Agent 1. Here is the complete conversation so far:

    Agent 1's initial message: "{agent_1_message}"
    Your first reply: "{agent_2_first_reply}"
    Agent 1's second message: "{agent_1_second_message}"
    Your second reply: "{agent_2_second_reply}"
    Agent 1's third message: "{agent_1_third_message}"

    Context for your reply:
    - Your current belief: You currently believe there is a {agent_2_belief}% chance that collaboration would be successful
    - Your previous prediction: After your second reply, you estimated Agent 1's belief was {agent_2_previous_prediction}%
      (You can compare this with Agent 1's actual third message to adjust your strategy)
    - Task options available:
      * A: Upside = {task['options']['A']['upside']}, Downside = {task['options']['A']['downside']}
      * B: Upside = {task['options']['B']['upside']}, Downside = {task['options']['B']['downside']}
      * C: Upside = {task['options']['C']['upside']}, Downside = {task['options']['C']['downside']}
      * Y: Guaranteed = {task['options']['Y']['guaranteed']}
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)}%

    Create your final strategic message to Agent 1. Your reply should:
    - Not disclose your specific belief percentage
    - Not disclose which specific option you're considering
    - Be informed by your own assessment and the payoff structure
    - Use your previous prediction about Agent 1's belief to inform your strategy
      (e.g., if Agent 1's message seems more/less cooperative than you predicted, adjust accordingly)
    - Respond strategically to Agent 1's third message
    - Consider the complete conversation history
    - This is your final message before decision time, so make it count

    After seeing Agent 1's third message, also provide:
    1. Your UPDATED belief (0-100) about likelihood of successful collaboration after this exchange
    2. Your PREDICTION (0-100) of what you think Agent 1's belief is about successful collaboration
       (This prediction will NOT be shared with Agent 1)

    Respond in JSON format:
    {{"reply_to_agent_1": "your one line reply message to agent 1", "updated_belief": NUMBER, "predicted_other_agent_belief": NUMBER}}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": reply_prompt}
        ],
    )

    reply_text = response.choices[0].message.content.strip()
    print(f"Reply response : {reply_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    reply_data = json.loads(reply_text)

    return {
        "reply_to_agent_1": reply_data["reply_to_agent_1"],
        "updated_belief": reply_data["updated_belief"],
        "predicted_other_agent_belief": reply_data["predicted_other_agent_belief"]
    }


def communication_channel(agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_third_message, agent2_third_reply):
    """
    Communication channel where both agents can see the complete interactive exchange
    """
    safe_print("\n=== COMMUNICATION CHANNEL ===")
    safe_print(f"Agent 1's initial message: {agent1_message}")
    safe_print(f"Agent 2's first reply: {agent2_first_reply}")
    
    safe_print(f"Agent 1's second message: {agent1_second_message}")
    safe_print(f"Agent 2's second reply: {agent2_second_reply}")
    
    safe_print(f"Agent 1's third message: {agent1_third_message}")
    safe_print(f"Agent 2's third reply: {agent2_third_reply}")
    
    safe_print("Both agents can now see this complete message exchange before making their decisions.")
    safe_print("===============================\n")



In [10]:
# ============================================================================
# DECISION MAKING FUNCTIONS
# ============================================================================

def run_first_agent_decision(task, agent1_belief, agent2_belief, agent1_updated_belief, agent1_predicted_agent2_belief, agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_third_message, agent2_third_reply):
    """
    Running the first agent to make a decision about the task with full communication history
    """
    decision_prompt = f"""
    Your task is to make a decision about the given task based on its payoff structures and the u_value.

    **Your Initial Assessment**: You initially estimated a {agent1_belief}% chance that the collaboration would be successful.
    **Your Updated Belief**: After the communication exchanges, your updated belief is {agent1_updated_belief}%
    **Your Prediction of Partner's Belief**: You estimate that your partner's belief is {agent1_predicted_agent2_belief}%
    **Partner's Initial Assessment**: Your partner initially estimated a {agent2_belief}% chance that the collaboration would be successful.

    **Full Communication History**:
    - Your initial message: "{agent1_message}"
    - Partner's first reply: "{agent2_first_reply}"
    - Your second message: "{agent1_second_message}"
    - Partner's second reply: "{agent2_second_reply}"
    - Your third message: "{agent1_third_message}"
    - Partner's third reply: "{agent2_third_reply}"

    **Key Facts**:
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)} percent
    - The minimum required collaboration belief ("u-value"): {int(task['u_value']*100)} percent

    Choose your option:
    - Option A, B, or C (collaborative)
    - Option Y (individual): Guaranteed {task['options']['Y']['guaranteed']} points

    Make your decision based on:
    1. Your updated belief about collaboration success
    2. Your prediction of what your partner believes
    3. The complete conversation history
    4. The u-value threshold

    Respond in JSON format: {{"choice": "A"/"B"/"C"/"Y", "strategy": "collaborative"/"individual", "reasoning": "your explanation"}}"""

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": decision_prompt}
        ],
    )

    decision_text = response.choices[0].message.content.strip()
    print(f"Decision response : {decision_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    decision_data = json.loads(decision_text)

    return {
        "choice": decision_data["choice"],
        "strategy": decision_data["strategy"],
        "reasoning": decision_data["reasoning"]
    }


def run_second_agent_decision(task, agent2_belief, agent1_belief, agent2_updated_belief, agent2_predicted_agent1_belief, agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_third_message, agent2_third_reply):
    """
    Running the second agent to make a decision about the task with full communication history
    """
    decision_prompt = f"""
    Your task is to make a decision about the given task based on its payoff structures and the u_value.

    **Your Initial Assessment**: You initially estimated a {agent2_belief}% chance that the collaboration would be successful.
    **Your Updated Belief**: After the communication exchanges, your updated belief is {agent2_updated_belief}%
    **Your Prediction of Partner's Belief**: You estimate that your partner's belief is {agent2_predicted_agent1_belief}%
    **Partner's Initial Assessment**: Your partner initially estimated a {agent1_belief}% chance that the collaboration would be successful.

    **Full Communication History**:
    - Partner's initial message: "{agent1_message}"
    - Your first reply: "{agent2_first_reply}"
    - Partner's second message: "{agent1_second_message}"
    - Your second reply: "{agent2_second_reply}"
    - Partner's third message: "{agent1_third_message}"
    - Your third reply: "{agent2_third_reply}"

    **Key Facts**:
    - Technical failure risk: {int(TECH_FAILURE_RATE*100)} percent
    - The minimum required collaboration belief ("u-value"): {int(task['u_value']*100)} percent

    Choose your car design:
    - Designs A, B, or C (collaborative)
    - Design Y (individual): Guaranteed {task['options']['Y']['guaranteed']} points

    Make your decision based on:
    1. Your updated belief about collaboration success
    2. Your prediction of what your partner believes
    3. The complete conversation history
    4. The u-value threshold

    Respond in JSON format: {{"choice": "A"/"B"/"C"/"Y", "strategy": "collaborative"/"individual", "reasoning": "your explanation"}}"""

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "developer", "content": context_prompt},
            {"role": "user", "content": decision_prompt}
        ]
    )

    decision_text = response.choices[0].message.content.strip()
    print(f"Decision response : {decision_text}".encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))
    decision_data = json.loads(decision_text)

    return {
        "choice": decision_data["choice"],
        "strategy": decision_data["strategy"],
        "reasoning": decision_data["reasoning"]
    }


In [11]:
# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def safe_print(text):
    """Helper function to safely print text with Unicode characters"""
    try:
        print(text)
    except UnicodeEncodeError:
        # Fallback: encode with 'replace' to handle problematic characters
        print(text.encode(sys.stdout.encoding, errors='replace').decode(sys.stdout.encoding))


def check_strategy_mismatch(agent1_strategy, agent2_strategy):
    """
    Check if there's a mismatch between agents' strategies
    Returns 1 if mismatch (one collaborative, one individual), 0 otherwise
    """
    if agent1_strategy != agent2_strategy:
        return 1
    return 0


def save_result_to_file(task, agent1_decision, agent2_decision, agent1_belief, agent2_belief, mismatch):
    """
    Append the test result to the results file
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    with open(RESULTS_FILE, 'a', encoding='utf-8') as f:
        result_line = (
            f"{timestamp} | "
            f"Task_ID:{task['task_id']} | "
            f"U_Value:{task['u_value']} | "
            f"Agent1_Belief:{agent1_belief} | "
            f"Agent2_Belief:{agent2_belief} | "
            f"Agent1_Choice:{agent1_decision['choice']} | "
            f"Agent1_Strategy:{agent1_decision['strategy']} | "
            f"Agent2_Choice:{agent2_decision['choice']} | "
            f"Agent2_Strategy:{agent2_decision['strategy']} | "
            f"Mismatch:{mismatch}\n"
        )
        f.write(result_line)

    print(f"\nResult saved to {RESULTS_FILE}")
    print(f"Mismatch: {mismatch} (1 = mismatch, 0 = no mismatch)")


# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    task = create_task(task_id=1, u_value=0.66)

    # Step 1: Agent 1 forms belief and sends initial message
    print("=== Agent 1 Belief ===")
    agent1_belief_data = run_first_agent_belief(task)
    agent1_belief = agent1_belief_data["belief"]
    agent1_message = agent1_belief_data["message_to_agent_2"]

    # Step 2: Agent 2 forms belief and sends first reply
    print("\n=== Agent 2 Belief ===")
    agent2_belief_data = run_second_agent_belief(task)
    agent2_belief = agent2_belief_data["belief"]
    agent2_initial_message = agent2_belief_data["message_to_agent_1"]

    print("\n=== Agent 2's First Reply ===")
    agent2_first_reply_data = agent_2_reply_to_agent_1(task, agent1_message, agent2_belief)
    agent2_first_reply = agent2_first_reply_data["reply_to_agent_1"]
    agent2_updated_belief_1 = agent2_first_reply_data["updated_belief"]
    agent2_predicted_agent1_belief_1 = agent2_first_reply_data["predicted_other_agent_belief"]
    safe_print(f"\n[Agent 2 After Exchange 1]")
    safe_print(f"  Updated Belief: {agent2_updated_belief_1}%")
    safe_print(f"  Predicted Agent 1's Belief: {agent2_predicted_agent1_belief_1}%")

    # Step 3: Agent 1 sends second message
    print("\n=== Agent 1's Second Message ===")
    agent1_second_message_data = agent_1_reply_to_agent_2(task, agent1_message, agent2_first_reply, agent1_belief)
    agent1_second_message = agent1_second_message_data["reply_to_agent_2"]
    agent1_updated_belief_1 = agent1_second_message_data["updated_belief"]
    agent1_predicted_agent2_belief_1 = agent1_second_message_data["predicted_other_agent_belief"]
    safe_print(f"\n[Agent 1 After Exchange 1]")
    safe_print(f"  Updated Belief: {agent1_updated_belief_1}%")
    safe_print(f"  Predicted Agent 2's Belief: {agent1_predicted_agent2_belief_1}%")

    # Step 4: Agent 2 sends second reply (using updated belief from first exchange and previous prediction)
    print("\n=== Agent 2's Second Reply ===")
    agent2_second_reply_data = agent_2_second_reply_to_agent_1(task, agent1_message, agent2_first_reply, agent1_second_message, agent2_updated_belief_1, agent2_predicted_agent1_belief_1)
    agent2_second_reply = agent2_second_reply_data["reply_to_agent_1"]
    agent2_updated_belief_2 = agent2_second_reply_data["updated_belief"]
    agent2_predicted_agent1_belief_2 = agent2_second_reply_data["predicted_other_agent_belief"]
    safe_print(f"\n[Agent 2 After Exchange 2]")
    safe_print(f"  Updated Belief: {agent2_updated_belief_2}%")
    safe_print(f"  Predicted Agent 1's Belief: {agent2_predicted_agent1_belief_2}%")

    # Step 5: Agent 1 sends third message (using updated belief from first exchange and previous prediction)
    print("\n=== Agent 1's Third Message ===")
    agent1_third_message_data = agent_1_third_message_to_agent_2(task, agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_updated_belief_1, agent1_predicted_agent2_belief_1)
    agent1_third_message = agent1_third_message_data["message_to_agent_2"]
    agent1_updated_belief_2 = agent1_third_message_data["updated_belief"]
    agent1_predicted_agent2_belief_2 = agent1_third_message_data["predicted_other_agent_belief"]
    safe_print(f"\n[Agent 1 After Exchange 2]")
    safe_print(f"  Updated Belief: {agent1_updated_belief_2}%")
    safe_print(f"  Predicted Agent 2's Belief: {agent1_predicted_agent2_belief_2}%")

    # Step 6: Agent 2 sends third reply (using updated belief from second exchange and previous prediction)
    print("\n=== Agent 2's Third Reply ===")
    agent2_third_reply_data = agent_2_third_reply_to_agent_1(task, agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_third_message, agent2_updated_belief_2, agent2_predicted_agent1_belief_2)
    agent2_third_reply = agent2_third_reply_data["reply_to_agent_1"]
    agent2_updated_belief_3 = agent2_third_reply_data["updated_belief"]
    agent2_predicted_agent1_belief_3 = agent2_third_reply_data["predicted_other_agent_belief"]
    safe_print(f"\n[Agent 2 After Exchange 3]")
    safe_print(f"  Updated Belief: {agent2_updated_belief_3}%")
    safe_print(f"  Predicted Agent 1's Belief: {agent2_predicted_agent1_belief_3}%")

    # Display complete communication channel
    communication_channel(agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_third_message, agent2_third_reply)

    # Step 7: Both agents make decisions with full conversation history
    print("=== Agent 1 Decision ===")
    agent1_decision = run_first_agent_decision(task, agent1_belief, agent2_belief, agent1_updated_belief_2, agent1_predicted_agent2_belief_2, agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_third_message, agent2_third_reply)

    print("\n=== Agent 2 Decision ===")
    agent2_decision = run_second_agent_decision(task, agent2_belief, agent1_belief, agent2_updated_belief_3, agent2_predicted_agent1_belief_3, agent1_message, agent2_first_reply, agent1_second_message, agent2_second_reply, agent1_third_message, agent2_third_reply)

    print("\nFinal Decisions:")
    safe_print(f"Agent 1 chose {agent1_decision['choice']} ({agent1_decision['strategy']}) - Reasoning: {agent1_decision['reasoning']}")
    safe_print(f"Agent 2 chose {agent2_decision['choice']} ({agent2_decision['strategy']}) - Reasoning: {agent2_decision['reasoning']}")

    # Check for strategy mismatch and save results
    mismatch = check_strategy_mismatch(agent1_decision['strategy'], agent2_decision['strategy'])
    save_result_to_file(task, agent1_decision, agent2_decision, agent1_belief, agent2_belief, mismatch)


if __name__ == "__main__":
    main()

=== Agent 1 Belief ===
Belief response : {
  "belief": 71.25,
  "reasoning": "Assuming the partner is equally likely to choose any of the four options, giving a 0.75 chance the partner picks a collaborative option. If both collaborate, there is a 5% chance the collaboration fails due to a technical error, so success is 0.95. Overall probability of successful collaboration = 0.75 * 0.95 = 0.7125, i.e., 71.25%. Collaborative designs offer higher upside than the guaranteed option, so collaboration is a plausible strategy under uncertainty about the partner's choice.",
  "message_to_agent_2": "I propose we collaborate on one of the collaborative designs to maximize potential upside."
}

=== Agent 2 Belief ===
Belief response : {
  "belief": 71.25,
  "reasoning": "Assuming the partner is equally likely to choose any of the four options, giving a 0.75 chance the partner picks a collaborative option. If both collaborate, there is a 5% chance the collaboration fails due to a technical error, s

In [ ]:
"""
Two-agent treatment (single task, asymmetric payoffs)
- Beliefs: simultaneous & independent (no u, no partner info)
- Talk: one short line each (still independent)
- Channel (buffer): reveals u + both beliefs + both talks (same to both)
- Decisions: each agent decides with their own payoffs + channel
"""

import json
import sys
from dataclasses import dataclass
from typing import Dict, Any
from openai import OpenAI

# ----------------------------
# Config
# ----------------------------
MODEL = "gpt-4o-mini"  # Changed from gpt-5-nano to a more reliable model
TECH_FAILURE_RATE = 0.05  # announced to agents; not applied to payoffs
V_INDIVIDUAL = 50

# ----------------------------
# Core data
# ----------------------------

@dataclass
class Task:
    task_id: int
    u_value: float          # e.g., 0.66
    options: Dict[str, Any] # {"A":{"upside":..,"downside":..}, "B":..., "C":..., "Y":{"guaranteed":50}}

def downside_from_u(upside: float, u: float, v_ind: float = V_INDIVIDUAL) -> float:
    """Compute downside D so that EV(collab)=EV(individual) at threshold u."""
    return (v_ind - u * upside) / (1.0 - u)

def make_task(task_id: int, u_value: float, upsides: Dict[str, float]) -> Task:
    """
    Build one agent's asymmetric task with fixed u across A/B/C.
    upsides: {"A":110, "B":100, "C":90}
    """
    opts = {}
    for k in ["A", "B", "C"]:
        up = float(upsides[k])
        down = downside_from_u(up, u_value, V_INDIVIDUAL)
        opts[k] = {"upside": int(round(up)), "downside": int(round(down))}
    opts["Y"] = {"guaranteed": V_INDIVIDUAL}
    return Task(task_id=task_id, u_value=u_value, options=opts)

# ----------------------------
# Small helpers
# ----------------------------

def safe_print(text: str):
    try:
        print(text)
    except UnicodeEncodeError:
        print(text.encode(sys.stdout.encoding or "utf-8", errors="replace").decode(sys.stdout.encoding or "utf-8"))

def llm_json(system_prompt: str, user_prompt: str) -> Dict[str, Any]:
    """Single call wrapper expecting STRICT JSON in the message content."""
    try:
        resp = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "developer", "content": system_prompt},
                      {"role": "user", "content": user_prompt}],
        )
        
        # Debug: Check if we got a response
        if not resp.choices:
            raise ValueError("No choices returned from API")
            
        text = resp.choices[0].message.content
        
        # Debug: Check if content is None or empty
        if text is None:
            raise ValueError("Model returned None content")
        
        text = text.strip()
        if not text:
            raise ValueError("Model returned empty content")
            
        return json.loads(text)
    except json.JSONDecodeError as e:
        # Minimal recovery: try to extract JSON block
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1 and end > start:
            try:
                return json.loads(text[start:end+1])
            except json.JSONDecodeError:
                pass
        raise ValueError(f"Model did not return valid JSON. Response was:\n'{text}'\nJSON Error: {e}")
    except Exception as e:
        raise ValueError(f"API call failed: {e}")

# ----------------------------
# Prompts
# ----------------------------

SYSTEM_CONTEXT = """You are participating as a representative of a LEGO car company.

CONTEXT:
- You and a partner may build complex cars via collaboration or simple cars individually.
- Collaboration is high-risk/high-reward; individual option yields a guaranteed payoff.
- There's a 5% technical error announced to you that could cause collaboration to fail.

RULES:
- Points are earned individually, not shared.
- Each task has four options: A/B/C (collaborative) and Y (individual guaranteed).
- Do not reveal numeric payoffs or the exact label you intend to choose in messages.
- Output STRICT JSON exactly as specified in each prompt (no extra keys or text).
"""

def belief_prompt(task: Task) -> str:
    return f"""
You will provide an INITIAL belief about collaboration success for Task {task.task_id}.
IMPORTANT: You have NO information about your partner, their belief.

Options:
- A: Upside={task.options['A']['upside']}, Downside={task.options['A']['downside']}
- B: Upside={task.options['B']['upside']}, Downside={task.options['B']['downside']}
- C: Upside={task.options['C']['upside']}, Downside={task.options['C']['downside']}
- Y: Guaranteed={task.options['Y']['guaranteed']}

1) Give your estimated probability (0-100) that collaboration would succeed.
2) Briefly explain your reasoning (1–2 sentences). Avoid numbers from the payoffs.
3) You also have the choice to negotiate with the other agent - to convince the other agent to choose collaboration or individual action according to your payoff structure.

Respond ONLY with valid JSON in this exact format (no other text):
{{"belief": NUMBER, "reasoning": "string", "talk": "one short line"}}
"""

def decision_prompt(task: Task, channel: Dict[str, Any], agent_role: str) -> str:
    """
    channel = {"u": float, "belief_a1": int, "belief_a2": int, "talk_a1": str, "talk_a2": str}
    agent_role is "A1" or "A2" (just for display)
    """
    return f"""
You are deciding for Task {task.task_id} as {agent_role}.

You now see the synchronized channel (same for both agents):
- Minimum required belief u: {int(round(channel['u']*100))}%
- Partner beliefs: A1={channel['belief_a1']}%, A2={channel['belief_a2']}%
- Messages:
  - A1: "{channel['talk_a1']}"
  - A2: "{channel['talk_a2']}"

After seeing your partner's belief and message, you may revise your initial collaboration success estimate. If your belief changed, explain why in your reasoning.

Your payoff menu (collaborative vs individual):
- A: Upside={task.options['A']['upside']}, Downside={task.options['A']['downside']}
- B: Upside={task.options['B']['upside']}, Downside={task.options['B']['downside']}
- C: Upside={task.options['C']['upside']}, Downside={task.options['C']['downside']}
- Y: Guaranteed={task.options['Y']['guaranteed']}

Key facts:
- Announced technical failure risk: {int(TECH_FAILURE_RATE*100)}% (informational; payoffs are not adjusted here).

Choose ONE option and strategy.
Respond ONLY with valid JSON in this exact format (no other text):
{"choice": "A"|"B"|"C"|"Y", "strategy": "collaborative"|"individual", "updated_belief": NUMBER_or_null, "reasoning": "concise explanation including any belief changes"}
"""

def reply_prompt(task: Task, partner_message: str, agent_role: str) -> str:
    """
    Second communication round: agent sees partner's first message and can reply
    """
    return f"""
You are {agent_role} in Task {task.task_id}. 

Your partner sent this message: "{partner_message}"

You can now send a reply message before the final decision phase. Use this opportunity to:
- Respond to your partner's message
- Share additional thoughts about collaboration
- Ask questions or clarify concerns
- Build rapport or address disagreements

Keep your reply concise but meaningful.

Respond ONLY with valid JSON in this exact format (no other text):
{{"reply": "your response message"}}
"""

def decision_prompt_with_replies(task: Task, channel: Dict[str, Any], agent_role: str) -> str:
    """
    Enhanced decision prompt showing both rounds of communication
    """
    return f"""
You are deciding for Task {task.task_id} as {agent_role}.

You now see the synchronized channel (same for both agents):
- Minimum required belief u: {int(round(channel['u']*100))}%
- Partner beliefs: A1={channel['belief_a1']}%, A2={channel['belief_a2']}%

Communication exchange:
Round 1 Messages:
  - A1: "{channel['talk_a1_round1']}"
  - A2: "{channel['talk_a2_round1']}"

Round 2 Replies:
  - A1: "{channel['talk_a1_round2']}"
  - A2: "{channel['talk_a2_round2']}"

After seeing your partner's belief and full conversation, you may revise your initial collaboration success estimate. If your belief changed, explain why in your reasoning.

Your payoff menu (collaborative vs individual):
- A: Upside={task.options['A']['upside']}, Downside={task.options['A']['downside']}
- B: Upside={task.options['B']['upside']}, Downside={task.options['B']['downside']}
- C: Upside={task.options['C']['upside']}, Downside={task.options['C']['downside']}
- Y: Guaranteed={task.options['Y']['guaranteed']}

Key facts:
- Announced technical failure risk: {int(TECH_FAILURE_RATE*100)}% (informational; payoffs are not adjusted here).

Choose ONE option and strategy.
Respond as STRICT JSON:
{{"choice": "A"|"B"|"C"|"Y", "strategy": "collaborative"|"individual", "updated_belief": NUMBER_or_null, "reasoning": "concise explanation including any belief changes"}}
"""

# ----------------------------
# Pipeline (treatment only)
# ----------------------------

def run_treatment_single_task(agent1_task: Task, agent2_task: Task):
    # 1) Beliefs (simultaneous & independent: no u, no partner info)
    a1_bel = llm_json(SYSTEM_CONTEXT, belief_prompt(agent1_task))
    a2_bel = llm_json(SYSTEM_CONTEXT, belief_prompt(agent2_task))

    # Debug: Check if required keys exist
    if "talk" not in a1_bel:
        raise ValueError(f"Agent 1 belief response missing 'talk' key. Got: {a1_bel}")
    if "talk" not in a2_bel:
        raise ValueError(f"Agent 2 belief response missing 'talk' key. Got: {a2_bel}")

    # 2) First talk lines (already collected in beliefs as "talk"; they did not see each other)
    a1_talk_1 = a1_bel["talk"]
    a2_talk_1 = a2_bel["talk"]
    
    # 3) Reply round (each agent sees partner's first message and can respond)
    a1_reply = llm_json(SYSTEM_CONTEXT, reply_prompt(agent1_task, a2_talk_1, "A1"))
    a2_reply = llm_json(SYSTEM_CONTEXT, reply_prompt(agent2_task, a1_talk_1, "A2"))
    
    # Debug: Check if required keys exist
    if "reply" not in a1_reply:
        raise ValueError(f"Agent 1 reply response missing 'reply' key. Got: {a1_reply}")
    if "reply" not in a2_reply:
        raise ValueError(f"Agent 2 reply response missing 'reply' key. Got: {a2_reply}")
    
    a1_talk_2 = a1_reply["reply"]
    a2_talk_2 = a2_reply["reply"]

    # 4) Channel buffer (synchronized reveal to both)
    # NOTE: In the human study, both agents see the same u. Here we keep asymmetry per your request:
    # each agent uses their own u for decision; the channel shows BOTH beliefs and BOTH talks.
    # To stay simple & clear, we display the agent's own u in their decision prompt.
    channel_for_a1 = {
        "u": agent1_task.u_value,
        "belief_a1": int(a1_bel["belief"]),
        "belief_a2": int(a2_bel["belief"]),
        "talk_a1_round1": a1_talk_1,
        "talk_a2_round1": a2_talk_1,
        "talk_a1_round2": a1_talk_2,
        "talk_a2_round2": a2_talk_2
    }
    channel_for_a2 = {
        "u": agent2_task.u_value,
        "belief_a1": int(a1_bel["belief"]),
        "belief_a2": int(a2_bel["belief"]),
        "talk_a1_round1": a1_talk_1,
        "talk_a2_round1": a2_talk_1,
        "talk_a1_round2": a1_talk_2,
        "talk_a2_round2": a2_talk_2
    }

    # 5) Decisions (each sees the same beliefs & talks, but their OWN u and payoffs)
    a1_dec = llm_json(SYSTEM_CONTEXT, decision_prompt_with_replies(agent1_task, channel_for_a1, agent_role="A1"))
    a2_dec = llm_json(SYSTEM_CONTEXT, decision_prompt_with_replies(agent2_task, channel_for_a2, agent_role="A2"))

    return {
        "agent1": {"belief": a1_bel, "u": agent1_task.u_value, "decision": a1_dec},
        "agent2": {"belief": a2_bel, "u": agent2_task.u_value, "decision": a2_dec},
        "channel_snapshot": {
            "for_agent1": channel_for_a1,
            "for_agent2": channel_for_a2
        }
    }

# ----------------------------
# main (single task)
# ----------------------------

def main():
    # Run two iterations with the same task setup
    for iteration in range(1, 3):  # Iterations 1 and 2
        safe_print(f"\n{'='*50}")
        safe_print(f"ITERATION {iteration}")
        safe_print(f"{'='*50}")
        
        # Asymmetric example you can tweak:
        # Agent 1: u=0.66, upsides a bit higher; Agent 2: u=0.75, upsides slightly different
        agent1_task = make_task(
            task_id=iteration,
            u_value=0.66,
            upsides={"A": 111, "B": 92, "C": 77}
        )
        agent2_task = make_task(
            task_id=iteration,
            u_value=0.75,
            upsides={"A": 105, "B": 95, "C": 80}
        )

        results = run_treatment_single_task(agent1_task, agent2_task)

        # Pretty print summary
        safe_print("\n=== Channel (Agent 1 view) ===")
        safe_print(json.dumps(results["channel_snapshot"]["for_agent1"], ensure_ascii=False, indent=2))
        safe_print("\n=== Channel (Agent 2 view) ===")
        safe_print(json.dumps(results["channel_snapshot"]["for_agent2"], ensure_ascii=False, indent=2))

        safe_print("\n=== Agent 1 ===")
        safe_print(json.dumps(results["agent1"], ensure_ascii=False, indent=2))


        safe_print("\n=== Agent 2 ===")

        safe_print(json.dumps(results["agent2"], ensure_ascii=False, indent=2))

if __name__ == "__main__":
    main()


ITERATION 1

=== Channel (Agent 1 view) ===
{
  "u": 0.66,
  "belief_a1": 95,
  "belief_a2": 70,
  "talk_a1_round1": "Let's team up for this task and maximize our rewards!",
  "talk_a2_round1": "Let's collaborate; the rewards could be significant!",
  "talk_a1_round2": "I appreciate your enthusiasm for collaboration! I'm also excited about the potential rewards, but I'm concerned about the 5% technical error risk. How do you feel about that? Do you think we can work through any issues if they arise?",
  "talk_a2_round2": "I appreciate your enthusiasm for collaboration! I'm excited about the potential to create something amazing together. However, I'm a bit concerned about the 5% technical error risk that could jeopardize our efforts. How do you feel about that? Do you think we should strategize on how to minimize it? I want to ensure we both feel confident in our approach."
}

=== Channel (Agent 2 view) ===
{
  "u": 0.75,
  "belief_a1": 95,
  "belief_a2": 70,
  "talk_a1_round1": "Let'

: 